# lunasol

In [1]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
from selenium import webdriver
import json
import platform
import re
import copy

# write to json file

In [2]:
def writeJSON(jsonString, output_name='data.json'):
    with open(output_name,'w',encoding='UTF-8') as file:
        file.write(jsonString)


# get item

In [9]:
def getItem(driver): #루나솔 버전
    html = driver.page_source
    soup = bs(html,'lxml')
    
    name = soup.find('div',{'class':'product_summary'}).h1.get_text()    
    url = driver.current_url
    category = soup.find('div',{'id':'directory_link'}).get_text().replace(' > '+name,"")
    imageList = []
    image_main = url_home + soup.find('div',{'class':'product_photo'}).img['src']
    imageList.append(image_main)
    
    
    # item 설정
    item = {'name':'#', 'url':'#', 'image':'#', 'color':'#', 'category':'#', 
                   'salePrice':'#', 'originalPrice':'#', 'brand':'lunasol','volume':'#'}
    
    item['name']=name
    item['url'] = url
    item['category']=category.strip()
    item['image'] = imageList
    
    # return 할 json 설정
    item_json = []
    
    # color로 상품 구분
    colors= soup.find('div',{'class':'product_color'})
    
    if colors: # 색상이 있는 경우
        colors = colors.find_all('img')
        for color in colors:
            image_color = url_home + color['src']
            name_color = color['alt']

            item_dict = copy.deepcopy(item)
            item_dict['image'].append(image_color)
            item_dict['color'] = name_color
            item_json.append(item_dict)
            
    else:
        colors='#'
        item_dict = copy.deepcopy(item)
        item_dict['color'] = colors
        item_json.append(item_dict)
        
    display(item_json)
    return item_json


# start

In [4]:
path = 'chromedriver.exe' if (platform.system() == 'Windows') else '/Users/jg/Desktop/develop/DataTeam/DataProcessing/product/crawling/chromedriver';
driver = webdriver.Chrome(path)

In [10]:
url_home = 'https://www.lunasol-net.com'
url_product = 'https://www.lunasol-net.com/kr/product/index.html'
driver.get(url_product)

## get category

In [11]:
html = driver.page_source
soup = bs(html,'lxml')
category_total = soup.find('div',{'class':'product_mn_list png_fix'})
category_big = category_total.find_all('div')

# crawling start

In [12]:
# open category, item
result = []
for categories in category_big:
    list_category = categories.find_all('li')
    for category in list_category:
        url_category = category.a['href'] # get url of category
        driver.get(url_home + url_category)
        
        html = driver.page_source
        soup = bs(html,'lxml')
        items = soup.find('div',{'class':'product_cat product_cat_first'})
        items = items.find_all('a')
        for item in items:
            driver.get(url_home + item['href'])
            item = getItem(driver)
            result += item
            
        driver.get(url_product)


[{'name': 'GEMINATE EYES N',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0101004.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/p0101004/img/l.jpg',
   'https://www.lunasol-net.com/kr/product/parts/p0101004/img/01_ce.jpg'],
  'color': '01 CE',
  'category': 'TOP > PRODUCT > POINT MAKEUP > EYE SHADOW',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'},
 {'name': 'GEMINATE EYES N',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0101004.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/p0101004/img/l.jpg',
   'https://www.lunasol-net.com/kr/product/parts/p0101004/img/05_rb.jpg'],
  'color': '05 RB',
  'category': 'TOP > PRODUCT > POINT MAKEUP > EYE SHADOW',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'}]

[{'name': 'THE BEIGE EYES',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0102022.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/p0102022/img/l.jpg',
   'https://www.lunasol-net.com/kr/product/parts/p0102022/img/01_neutral_beige.jpg'],
  'color': '01 Neutral Beige',
  'category': 'TOP > PRODUCT > POINT MAKEUP > EYE SHADOW',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'},
 {'name': 'THE BEIGE EYES',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0102022.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/p0102022/img/l.jpg',
   'https://www.lunasol-net.com/kr/product/parts/p0102022/img/02_noble_beige.jpg'],
  'color': '02 Noble Beige',
  'category': 'TOP > PRODUCT > POINT MAKEUP > EYE SHADOW',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'}]

[{'name': 'NUANCE SHADE EYES',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0101033.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/p0101033/img/l.jpg',
   'https://www.lunasol-net.com/kr/product/parts/p0101033/img/01_romantic_scene.jpg'],
  'color': '01 Romantic Scene',
  'category': 'TOP > PRODUCT > POINT MAKEUP > EYE SHADOW',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'},
 {'name': 'NUANCE SHADE EYES',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0101033.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/p0101033/img/l.jpg',
   'https://www.lunasol-net.com/kr/product/parts/p0101033/img/02_tender_scene.jpg'],
  'color': '02 Tender Scene',
  'category': 'TOP > PRODUCT > POINT MAKEUP > EYE SHADOW',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'},
 {'name': 'NUANCE SHADE EYES',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointm

[{'name': 'MACARON GLOW EYES',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0101031.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/p0101031/img/l.jpg',
   'https://www.lunasol-net.com/kr/product/parts/p0101031/img/01_cassis.jpg'],
  'color': '01 Cassis',
  'category': 'TOP > PRODUCT > POINT MAKEUP > EYE SHADOW',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'},
 {'name': 'MACARON GLOW EYES',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0101031.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/p0101031/img/l.jpg',
   'https://www.lunasol-net.com/kr/product/parts/p0101031/img/02_green_tea.jpg'],
  'color': '02 Green Tea',
  'category': 'TOP > PRODUCT > POINT MAKEUP > EYE SHADOW',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'},
 {'name': 'MACARON GLOW EYES',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0101031.html',
  

[{'name': 'SHINE FALL EYES',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0101030.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/p0101030/img/l.jpg',
   'https://www.lunasol-net.com/kr/product/parts/p0101030/img/01_natural.jpg'],
  'color': '01 Natural',
  'category': 'TOP > PRODUCT > POINT MAKEUP > EYE SHADOW',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'},
 {'name': 'SHINE FALL EYES',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0101030.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/p0101030/img/l.jpg',
   'https://www.lunasol-net.com/kr/product/parts/p0101030/img/02_sophisticate.jpg'],
  'color': '02 Sophisticate',
  'category': 'TOP > PRODUCT > POINT MAKEUP > EYE SHADOW',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'},
 {'name': 'SHINE FALL EYES',
  'url': 'https://www.lunasol-net.com/kr/product/details/pointmake/p0101030.html',


...

[{'name': 'OIL-IN SOLUTION',
  'url': 'https://www.lunasol-net.com/kr/product/details/skincare/s0101008.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/s0101008/img/l.jpg'],
  'color': '#',
  'category': 'TOP > PRODUCT > SKINCARE',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'}]

[{'name': 'POSITIVE SOLUTION',
  'url': 'https://www.lunasol-net.com/kr/product/details/skincare/s0101009.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/s0101009/img/l.jpg'],
  'color': '#',
  'category': 'TOP > PRODUCT > SKINCARE',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'}]

[{'name': 'WATERY CLEAR OIL CLEANSING',
  'url': 'https://www.lunasol-net.com/kr/product/details/skincare/s0101006.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/s0101006/img/l.jpg'],
  'color': '#',
  'category': 'TOP > PRODUCT > SKINCARE',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'}]

[{'name': 'GLOWING DAY CREAM',
  'url': 'https://www.lunasol-net.com/kr/product/details/skincare/s0101007.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/s0101007/img/l.jpg'],
  'color': '#',
  'category': 'TOP > PRODUCT > SKINCARE',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'}]

[{'name': 'WRAPPING SKIN REPAIR',
  'url': 'https://www.lunasol-net.com/kr/product/details/skincare/s0201007.html',
  'image': ['https://www.lunasol-net.com/kr/product/parts/s0201007/img/l.jpg'],
  'color': '#',
  'category': 'TOP > PRODUCT > SKINCARE',
  'salePrice': '#',
  'originalPrice': '#',
  'brand': 'lunasol',
  'volume': '#'}]

In [13]:
output = json.dumps(result,ensure_ascii=False, indent='\t')

writeJSON(output, output_name = 'lunasol.json')